In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install openpyxl

In [ ]:
import os
import re
import time
import spm1d
import itertools
import openpyxl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy as sio
import scipy.io as sio
import multiprocessing as mp

from scipy import interpolate
from scipy import signal
from scipy.signal import savgol_filter
from joblib import Parallel, delayed
from lolviz import *
from pylab import *

print('libs imported')

In [ ]:
# Define functions
def fitF(data):
    datalist = []
    for c in range(1,180):
        strain = data.Strain[data.Cycle == c+0.5]
        stress = data.Stress[data.Cycle == c+0.5]
        if strain.empty:
            datalist.append([np.nan,np.nan,np.nan,np.nan])
        else:
            fit = np.polyfit(strain-min(strain), np.log(stress), 1, w=np.sqrt(stress),full = True)
            datalist.append([c,fit[0][0],exp(fit[0][1]),fit[1][0]])
    output = pd.DataFrame(datalist, columns=['Cycle','One','Two','Residual'])
    averages = [output.One.mean(),output.Two.mean(),output.Residual.mean(),output.One.std(),output.Two.std(),output.Residual.std()]
    return output, averages

def Average(lst): 
    return sum(lst) / len(lst) 

print('functinos defined')

In [ ]:
# Get raw data from MTS files and insert into dataframe
parameters = pd.read_excel(os.getcwd() + '/Data/Parameters.xlsx')
fitlist = []
datalist = []
exp1Alist = []
exp2Alist = []
residualAlist = []
exp1SDlist = []
exp2SDlist = []
residualSDlist = []
for folder,height,area in zip(parameters.key,parameters.height,parameters.area):
    data = pd.read_csv(os.getcwd() + '/Data/'+ folder + '/DAQ- Time, … - (Timed).txt',sep='\t',skiprows=[0,1,2,3,4,5,7])
    data.columns = ['Time', 'Displacement', 'Force', 'Error','Cycle']
    minX = max(data.Displacement[data.Cycle == 0.5])
    Displacement_Norm = []
    Strain = []
    Stress = []
    Strain_True = []
    Stress_True = []
    for x,y in zip(data.Displacement,data.Force):
        Displacement_Norm.append(x-minX)
        newx = -(x-minX)/height
        Strain.append(newx)
        newy = -y/area
        Stress.append(newy)
        Strain_True.append(-np.log(1-newx))
        Stress_True.append(newy*(1-newx))
    data.insert(5,"Displacement_Norm",Displacement_Norm)
    data.insert(6,"Strain",Strain)
    data.insert(7,"Stress",Stress)
    data.insert(8,"Strain_True",Strain_True)
    data.insert(9,"Stress_True",Stress_True)
    datalist.append(data)
    fit, averages = fitF(data)
    exp1Alist.append(averages[0])
    exp2Alist.append(averages[1])
    residualAlist.append(averages[2])
    exp1SDlist.append(averages[3])
    exp2SDlist.append(averages[4])
    residualSDlist.append(averages[5])
    fitlist.append(fit)
parameters.insert(7,"Data",datalist)
parameters.insert(8,"ExpAvg",exp1Alist)
parameters.insert(9,"OffsetAvg",exp2Alist)
parameters.insert(10,"ResidualAvg",residualAlist)
parameters.insert(11,"ExpSD",exp1SDlist)
parameters.insert(12,"OffsetSD",exp2SDlist)
parameters.insert(13,"ResidualSD",residualSDlist)
parameters.insert(14,"Fit",residualSDlist)

parameters.head()

In [ ]:
export = pd.concat([parameters['key'], parameters['state'],parameters['timepoint'],parameters['sample'],parameters['state name'],
                   parameters['ExpAvg'], parameters['OffsetAvg'],parameters['ResidualAvg'],parameters['ExpSD'],parameters['OffsetSD'],parameters['ResidualSD']],
                   axis=1,
                   keys=['key','state','timepoint','sample','state name','ExpAvg','OffsetAvg','ResidualAvg','ExpSD','OffsetSD','ResidualSD'])
export.to_excel('output.xlsx')

In [ ]:

#polynomial fit
# timepoints = [0,1,2,3,4]
# statenames = ['intact','damaged','repaired']
# samples = [11,12,13,14,15,21,22,23,24,25,26,31,32,33,41,42,43,44,45,51,52,53,54,55]
# results = []
# for sample in samples:
#     for timepoint in timepoints:
#         if timepoint > 0:
#             before = parameters.Fit[(parameters['timepoint']==timepoint) & (parameters['sample']==sample)]
#             after = parameters.Fit[(parameters['timepoint']==(timepoint-1)) & (parameters['sample']==sample)]
#             state = parameters['state name'][(parameters['timepoint']==timepoint) & (parameters['sample']==sample)].values[0]
#             valC = []
#             for valB,valA in zip(before,after):
#                 valC.append((valB.Three-valA.Three)/valA.Three)
#             results.append([sample,timepoint,state] + valC[0].values.tolist())
# listofnames = ['sample','timepoint','state']+ [n for n in range(179)]
# threedf = pd.DataFrame(results,columns=listofnames)

timepoints = [0,1,2,3,4]
statenames = ['intact','damaged','repaired']
samples = [11,12,13,14,15,21,22,23,24,25,26,31,32,33,41,42,43,44,45,51,52,53,54,55]
results = []
for sample in samples:
    for timepoint in timepoints:
        if timepoint > 0:
            after = parameters.Fit[(parameters['timepoint']==timepoint) & (parameters['sample']==sample)]
            before = parameters.Fit[(parameters['timepoint']==(timepoint-1)) & (parameters['sample']==sample)]
            state = parameters['state name'][(parameters['timepoint']==timepoint) & (parameters['sample']==sample)].values[0]
            valC = []
            for valB,valA in zip(after,before):
                valC.append((valB.Two-valA.Two)/valA.Two)
            results.append([sample,timepoint,state] + valC[0].values.tolist())
listofnames = ['sample','timepoint','state']+ [n for n in range(179)]
twodf = pd.DataFrame(results,columns=listofnames)

timepoints = [0,1,2,3,4]
statenames = ['intact','damaged','repaired']
samples = [11,12,13,14,15,21,22,23,24,25,26,31,32,33,41,42,43,44,45,51,52,53,54,55]
results = []
for sample in samples:
    for timepoint in timepoints:
        if timepoint > 0:
            after = parameters.Fit[(parameters['timepoint']==timepoint) & (parameters['sample']==sample)]
            before = parameters.Fit[(parameters['timepoint']==(timepoint-1)) & (parameters['sample']==sample)]
            state = parameters['state name'][(parameters['timepoint']==timepoint) & (parameters['sample']==sample)].values[0]
            valC = []
            for valB,valA in zip(after,before):
                valC.append((valB.One-valA.One)/valA.One)
            results.append([sample,timepoint,state] + valC[0].values.tolist())
listofnames = ['sample','timepoint','state']+ [n for n in range(179)]
onedf = pd.DataFrame(results,columns=listofnames)

In [ ]:
# Single variable analysis
timepoint = 1
state = ['intact','damaged']
length = range(1,179)
samplesN = len(threedf[0][(threedf.timepoint == timepoint) & (threedf.state == state[0])])

In [ ]:
# Multivariate analysis
timepoint = 1
state = ['intact','damaged']
length = range(1,179)
samplesN = len(threedf[0][(threedf.timepoint == timepoint) & (threedf.state == state[0])])

YA = np.zeros(shape=(samplesN,len(length),2))
for m in range(samplesN):
    for n,n1 in zip(length,range(len(length))):
        YA[m,n1] = np.array([onedf[n][(onedf.timepoint == timepoint) & (onedf.state == state[0])].values[m],
                             twodf[n][(twodf.timepoint == timepoint) & (twodf.state == state[0])].values[m]])
#                             threedf[n][(threedf.timepoint == timepoint) & (threedf.state == state[0])].values[m]])
        
YB = np.zeros(shape=(samplesN,len(length),2))
for m in range(samplesN):
    for n,n1 in zip(length,range(len(length))):
        YB[m,n1] = np.array([onedf[n][(onedf.timepoint == timepoint) & (onedf.state == state[1])].values[m],
                             twodf[n][(twodf.timepoint == timepoint) & (twodf.state == state[1])].values[m]])
#                             threedf[n][(threedf.timepoint == timepoint) & (threedf.state == state[1])].values[m]])



#(1) Conduct test:
alpha        = 0.05
T2           = spm1d.stats.hotellings_paired(YA, YB)
T2i          = T2.inference(0.05)
print(T2i)


#(2) Plot:
plt.close('all')
rcParams["figure.figsize"] = [20, 10] 
ax0     = plt.subplot(221)
ax1     = plt.subplot(222)
ax3     = plt.subplot(224)
## plot SPM results:
ax0.plot(YA[:,:,0].T, 'k', label='slow')
ax0.plot(YB[:,:,0].T, 'r', label='fast')
ax1.plot(YA[:,:,1].T, 'k')
ax1.plot(YB[:,:,1].T, 'r')
T2i.plot(ax=ax3)
plt.show()

In [ ]:
results=[]
for timepoint in timepoints:
    if timepoint > 0:
        for state in statenames:
            vals = []
            for n in range(179):
                vals.append(threedf[n][(threedf.timepoint == timepoint) & (threedf.state == state)].mean())
            results.append([timepoint,state] + vals)
listofnames = ['timepoint','state'] + [n for n in range(179)]
three_results = pd.DataFrame(results,columns = listofnames)
three_results.head()

In [ ]:
results=[]
for timepoint in timepoints:
    if timepoint > 0:
        for state in statenames:
            vals = []
            for n in range(179):
                vals.append(twodf[n][(twodf.timepoint == timepoint) & (twodf.state == state)].mean())
            results.append([timepoint,state] + vals)
listofnames = ['timepoint','state'] + [n for n in range(179)]
two_results = pd.DataFrame(results,columns = listofnames)
two_results.head()

In [ ]:
results=[]
for timepoint in timepoints:
    if timepoint > 0:
        for state in statenames:
            vals = []
            for n in range(179):
                vals.append(onedf[n][(onedf.timepoint == timepoint) & (onedf.state == state)].mean())
            results.append([timepoint,state] + vals)
listofnames = ['timepoint','state'] + [n for n in range(179)]
one_results = pd.DataFrame(results,columns = listofnames)
one_results.head()

In [ ]:
toplot1 = []
toplot2 = []
toplot3 = []
for n in range (2,179):
    toplot1.append(three_results[(three_results.state == 'intact') & (three_results.timepoint == 4)][n].values[0])
    toplot2.append(three_results[(three_results.state == 'damaged') & (three_results.timepoint == 4)][n].values[0])
    toplot3.append(three_results[(three_results.state == 'repaired') & (three_results.timepoint == 4)][n].values[0])
plot(range(2,179),toplot1)
plot(range(2,179),toplot2)
plot(range(2,179),toplot3)

In [ ]:
plt.rcParams["figure.figsize"] = [15, 10]
for name,sample in zip(parameters.timepoint[(parameters['state name'] == 'intact')],parameters.Fit[(parameters['state name'] == 'intact')]):
    plt.scatter(sample.Cycle,sample.Three,s=1,label=name)
    plt.legend()

In [ ]:
#strain0 = parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 1.5] - min(parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 1.5])
#plt.scatter(strain0,output[0][1](strain0),color='red')
#plt.scatter(strain0,parameters.Data[0].Stress_True[parameters.Data[0].Cycle == 1.5],color='blue')
strain1 = parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 140.5]-min(parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 140.5])
plt.scatter(strain1,output[139][1](strain1),color='red')
plt.scatter(strain1,parameters.Data[0].Stress_True[parameters.Data[0].Cycle == 140.5],color='blue')
strain2 = parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 130.5]-min(parameters.Data[0].Strain_True[parameters.Data[0].Cycle == 130.5])
plt.scatter(strain2,output[129][1](strain2),color='red')
plt.scatter(strain2,parameters.Data[0].Stress_True[parameters.Data[0].Cycle == 130.5],color='blue')